In [9]:
#!/opt/local/bin/python

import MySQLdb
import networkx as nx
import matplotlib.pylab as plt
from geopy.distance import great_circle
from geopy.geocoders import Nominatim 
import folium

# DEFINE SQL LOG IN 
HOST = 'localhost'
USER = 'root'
PASSWD = ''
DATABASE = 'bk_map'

#CONNECT TO MySQL
db_connect = MySQLdb.connect(
	host = HOST,
	user = USER,
	passwd = PASSWD,
	db = DATABASE)

cur = db_connect.cursor()

def nearest_node(lat_input, lon_input):
    dis_near=0.05
    find_nearby_nodes = ('''SELECT nodeid, latitude, longitude FROM bk_nodes 
                WHERE latitude BETWEEN %s AND %s
                AND longitude BETWEEN %s AND %s
                ''')
    node_range = (lat_input-dis_near, lat_input+dis_near, lon_input-dis_near, lon_input+dis_near)

    cur.execute(find_nearby_nodes,node_range)  
    rows = cur.fetchall()
    if len(rows) == 0:
        return 'NULL'
    #for row in rows:
    else:
        dist = [rows[0][0],great_circle((lat_input,lon_input),rows[0][1:3]).miles ]
        for row in rows[1:]:
            node_dist =  great_circle((lat_input,lon_input),row[1:3]).miles
            if dist[1] > node_dist:
                dist = [row[0],node_dist]
        return dist[0]

def gen_network(node_dic, edge_dic):
	G = nx.Graph()
	for d in node_dic:
 		G.add_node(d)
	
	for e in edge_dic:
		G.add_edge(str(edge_dic[e][0]),str(edge_dic[e][1]), weight = edge_dic[e][2])

	#print 'NODES: ', G.number_of_nodes()
	#print 'EDGES: ', G.number_of_edges()
	return G

def ReadNodefromSQL():
	#fetch data for nodes and edges
	cur.execute('''SELECT nodeid, latitude, longitude FROM bk_nodes
                ''')  
	nodes_sql = cur.fetchall()
	return nodes_sql

def ReadEdgefromSQL():
	cur.execute('''SELECT edgeid, node1, node2, weight FROM bk_weights_acc_nn
                ''')  
	edge_sql = cur.fetchall()
	return edge_sql

def NodeSQLtoDic(nodes_sql):
	node_dic = {}
	for node in nodes_sql:
		node_dic[str(node[0])] = str(node[1]) ,str(node[2]) 
	return node_dic

def EdgeDQLtoDic(edge_sql):
	edge_dic = {}
	for edge in edge_sql:
		edge_dic[edge[0]]= edge[1:]
	return edge_dic


def make_path(start_lat, start_lng, end_lat, end_lng, G):
#INPUT 
	start = nearest_node(start_lat,start_lng)
	end = nearest_node(end_lat, end_lng)
#exeption!!
	#nx.has_path(G,start,end)
	return nx.shortest_path(G,source = start, target = end, weight='weight'), nx.shortest_path_length(G,source = start, target = end, weight='weight')#
	 

def plot_path(node_dic, path_nodes):
	for d in node_dic:
		plt.scatter(node_dic[d][0],node_dic[d][1])
    
	node_list = []

	for p in path_nodes[0]:
		node_list.append(node_dic[str(p)]) 
            
		plt.plot(*zip(*node_list), color='g')

	node_list = []
	for p in path_nodes[1]:
		node_list.append(node_dic[str(p)])

		plt.plot(*zip(*node_list), color='r')

	plt.show()
	return

def crawl_path(start_lat, start_lng, bar_lat, bar_lng, end_lat, end_lng):
	path1 = make_path(start_lat, start_lng, bar_lat, bar_lng)
	path2 = make_path(bar_lat, bar_lng, end_lat, end_lng)
	return path1, path2

def close_db():
	cur.close()
	db_connect.close()

#ACTUAL PROGRAM



geolocator = Nominatim()

location = geolocator.geocode("Atlantic Terminal, Brooklyn")
start_lat, start_lng = location.latitude, location.longitude


location = geolocator.geocode("82 4th Ave, Brooklyn")
bar_lat, bar_lng = location.latitude, location.longitude

location = geolocator.geocode("218 Myrtle Ave, Brooklyn")
end_lat, end_lng = location.latitude, location.longitude

#max_lat = max([start_lat,bar_lat, end_lat])
#max_lng = max([start_lng,bar_lng, end_lng])

#min_lat = min([start_lat,bar_lat, end_lat])
#min_lng = min([start_lng,bar_lng, end_lng])

#print min_lat, max_lat
#print min_lng, max_lng

node_sql = ReadNodefromSQL()
edge_sql = ReadEdgefromSQL()

node_dic = NodeSQLtoDic(node_sql)
edge_dic = EdgeDQLtoDic(edge_sql)
G = gen_network(node_dic, edge_dic)



#start_lat = 40.6868 
#start_lng = -73.9824
#bar_lat =40.68200000 
#bar_lng = -73.98020000
#end_lat = 40.67430000 
#end_lng = -73.99540000

#path = make_path(start_lat, start_lng, end_lat, end_lng)
path = crawl_path(start_lat, start_lng, bar_lat, bar_lng, end_lat, end_lng)

TypeError: make_path() takes exactly 5 arguments (4 given)

In [ ]:
cur.execute("""SELECT id, latitude, longitude , n_ped_inj  
    FROM bk_trafficaccidents 
    WHERE n_ped_inj > 0
    AND date BETWEEN '2014-09-01' AND '2015-09-01'
    """)
accidents_sql = cur.fetchall()



In [116]:
accidents_sql[1]
accidents_dic = {}
accident_loc = []
for accident in accidents_sql:
    accidents_dic[accident[0]] = accident[1:]
    accident_loc.append(accident[1:3])
#from sklearn.neighbors import KernelDensity

In [117]:
#accidents_dic

line_paths = []
for pa in path:
    line = []
    for node in pa:
        line.append([float(node_dic[node][0]), float(node_dic[node][1])])
    line_paths.append(line)
        
print line_paths

[[[40.684052, -73.977457], [40.6843767, -73.978278], [40.6837417, -73.978684], [40.6831067, -73.979106], [40.6824707, -73.9795607], [40.6825127, -73.9796725], [40.6818815, -73.9800945]], [[40.6818815, -73.9800945], [40.6812492, -73.980523], [40.6806157, -73.9809537], [40.6799801, -73.9813841], [40.68088, -73.983711], [40.681642, -73.985672], [40.68228, -73.985231], [40.682909, -73.984797], [40.683543, -73.984375], [40.684389, -73.986564], [40.685024, -73.9861337], [40.68566, -73.985689], [40.686504, -73.987859], [40.687365, -73.990035], [40.687959, -73.991509], [40.688571, -73.991094], [40.689135, -73.9906813], [40.6890931, -73.9905478], [40.6894849, -73.9902371], [40.6896947, -73.9900635], [40.6903275, -73.9896421], [40.6909681, -73.9892198], [40.6920881, -73.988932], [40.6923017, -73.9889003], [40.6924248, -73.9888886], [40.6928067, -73.9887616], [40.6931979, -73.9887289], [40.6933876, -73.9887173], [40.6944204, -73.9886543], [40.6950184, -73.9886116], [40.696094, -73.9885539], [40.6

In [66]:
#custom = folium.Map(location=[45.5236, -122.6750], tiles='Mapbox', API_key='wrobstory.map-12345678')

map_osm = folium.Map(location=[40.6928, -73.990340], width=500, height = 300)
map_osm.polygon_marker(location=[start_lat, start_lng], popup='Starting Point',
                     fill_color='#132b5e', num_sides=3, radius=10)
map_osm.polygon_marker(location=[bar_lat, bar_lng], popup='Bar',
                     fill_color='#45647d', num_sides=4, radius=10)
map_osm.polygon_marker(location=[end_lat, end_lng], popup='Ending Point',
                     fill_color='#769d96', num_sides=6, radius=10)

for accident in accident_loc:
    #print accident
    map_osm.circle_marker(location=accident, fill_color='#000000', radius=2)
    
for path in line_paths:
    map_osm.line(path, line_color='#FF0000', line_weight=5)
    
map_osm.create_map(path='accidents1.html')


In [80]:
map_osm = folium.Map(location=[40.684901, -73.977680], width=500, height = 300, zoom_start=14)
for node in node_dic:
    map_osm.circle_marker(location=node_dic[node], fill_color='#000000', radius=1)
map_osm.create_map(path='nodes.html')

In [118]:
map_osm = folium.Map(location=[40.684901, -73.977680], width=500, height = 300, zoom_start=14)
for accident in accident_loc:
    #print accident
    map_osm.circle_marker(location=accident, line_color='#FF3300', radius=3)
map_osm.create_map(path='justaccidents.html')

In [64]:
def path_node_line(path, node_dic):
    line_paths = []
    for pa in path:
        line=[]
        for node in pa:
            line.append([node_dic[node][0],node_dic[node][1]])
        line_paths.append(line)
    return line_paths

In [113]:
path = crawl_path(start_lat, start_lng, bar_lat, bar_lng, end_lat, end_lng)
#path_node_list(path[1], node_dic)
line_paths = []
for pa in path:
    line=[]
    for node in pa:
        line.append([float(node_dic[node][0]),float(node_dic[node][1])])
    line_paths.append(line)
print line_paths

[[[40.684052, -73.977457], [40.6843767, -73.978278], [40.6837417, -73.978684], [40.6831067, -73.979106], [40.6824707, -73.9795607], [40.6825127, -73.9796725], [40.6818815, -73.9800945]], [[40.6818815, -73.9800945], [40.6812492, -73.980523], [40.6806157, -73.9809537], [40.6799801, -73.9813841], [40.68088, -73.983711], [40.681642, -73.985672], [40.68228, -73.985231], [40.682909, -73.984797], [40.683543, -73.984375], [40.684389, -73.986564], [40.685024, -73.9861337], [40.68566, -73.985689], [40.686504, -73.987859], [40.687365, -73.990035], [40.687959, -73.991509], [40.688571, -73.991094], [40.689135, -73.9906813], [40.6890931, -73.9905478], [40.6894849, -73.9902371], [40.6896947, -73.9900635], [40.6903275, -73.9896421], [40.6909681, -73.9892198], [40.6920881, -73.988932], [40.6923017, -73.9889003], [40.6924248, -73.9888886], [40.6928067, -73.9887616], [40.6931979, -73.9887289], [40.6933876, -73.9887173], [40.6944204, -73.9886543], [40.6950184, -73.9886116], [40.696094, -73.9885539], [40.6

In [120]:
close_db()

In [82]:
from datetime import datetime
startTime = datetime.now()

#start_lat = 40.6868 
#start_lng = -73.9824
#bar_lat =40.68200000 
#bar_lng =  -73.98020000
#end_lat = 40.67430000 
#end_lng = -73.99540000

start_tuple = (40.6868, -73.9824)
bar_tuple_list = [( 40.68680000, -73.97510000),(40.68200000, -73.98020000),( 40.69340000, -73.97300000), ( 40.69330000,-73.96910000),(40.67720000, -73.95740000)]
end_tuple = (40.67430000,-73.99540000)
#end_tuple = start_tuple
#generate graph including points and locations
#define location dictionary of crawl
crawl = {}
crawl['S'] = start_tuple 
if start_tuple == end_tuple:
    start = 'S'
    end ='S'
    pass
elif end_tuple!=start_tuple:
    crawl['E'] = end_tuple
    start = 'S'
    end = 'E'
i=1
for bar in bar_tuple_list:
    crawl['B'+str(i)]= bar
    i+=1
#print crawl
#loop through all combinations of nodes to find distances
crawl_edge = {}
for loc1 in crawl:
    for loc2 in crawl:
        if loc1 == loc2:
            pass
        else:
            #print crawl[loc1][0], crawl[loc1][1], crawl[loc2][0],crawl[loc2][1]
            dist = make_path (crawl[loc1][0], crawl[loc1][1], crawl[loc2][0],crawl[loc2][1],G)
            crawl_edge[loc1+loc2] = loc1, loc2, dist[1], dist[0]
            
G_crawl = gen_network(crawl, crawl_edge)

distances = []


for path in nx.all_simple_paths(G_crawl, source= start , target = end, cutoff = len(bar_tuple_list)+2):
    #we only want the longest ones
    if(len(path)==len(bar_tuple_list)+2):
        distance=0
        #add up the weights for each edge
        for i in range(0,len(path)-1):
            distance=distance+G_crawl[path[i]][path[i+1]]['weight']
            #add these to a list
        distances.append((distance,path))
            
#defaults
min_dist=distances[0];
min_path=[]

#find the min distance
for dist in distances:
    if(dist[0]<=min_dist):
        min_dist=dist[0]
        min_path=dist[1]


route = []
for i in range(0,len(min_path)-1):
    route.append(crawl_edge[min_path[i]+min_path[i+1]][3])

line_paths = path_node_line(route, node_dic)


print(min_path)
print("has a distance of")
print(min_dist)
#print (route) #nodes
print (line_paths) #lat and lng of nodes

print datetime.now() - startTime

['S', 'B1', 'B3', 'B4', 'B5', 'B2', 'E']
has a distance of
6.76138288
[[['40.686721', '-73.982232'], ['40.685959', '-73.980272'], ['40.686641', '-73.9798125'], ['40.6866757', '-73.9793577'], ['40.686713', '-73.979057'], ['40.68682', '-73.978208'], ['40.686919', '-73.977361'], ['40.686994', '-73.976545'], ['40.687025', '-73.976206'], ['40.686757', '-73.97552']], [['40.686757', '-73.97552'], ['40.687086', '-73.97559'], ['40.689737', '-73.976102'], ['40.689698', '-73.975106'], ['40.6899975', '-73.9750673'], ['40.6900944', '-73.9747461'], ['40.6904598', '-73.9746921'], ['40.6908661', '-73.9746941'], ['40.6911358', '-73.9745235'], ['40.6914294', '-73.9740166'], ['40.6917117', '-73.9739184'], ['40.691707', '-73.97354'], ['40.69168', '-73.972553'], ['40.6932527', '-73.972851'], ['40.6935217', '-73.972919']], [['40.6935217', '-73.972919'], ['40.6932527', '-73.972851'], ['40.69321', '-73.971872'], ['40.693172', '-73.970859'], ['40.693135', '-73.969831'], ['40.693114', '-73.9688092']], [['40.693

[('S', 'E'), ('S', 'B1'), ('S', 'B2'), ('E', 'B1'), ('E', 'B2'), ('B1', 'B2')]

In [83]:
map_osm = folium.Map(location=[40.6928, -73.990340], width=500, height = 300)

for location in crawl:
    map_osm.polygon_marker(location=[crawl[location][0], crawl[location][1]], popup=location,
                     fill_color='#132b5e', num_sides=3, radius=10)

for path in line_paths:
    map_osm.line(path, line_color='#FF0000', line_weight=5)
    
map_osm.create_map(path='salesman.html')
